In [4]:
"""
BERTの勉強 note3
"""
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from transformers import AutoModel, AutoTokenizer
from transformers import TrainingArguments, Trainer

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support

from tqdm import tqdm
import glob, pickle

pretrained_model_name = "cl-tohoku/bert-base-japanese"

In [5]:
# タスク用Datasetクラスを定義
class LivedoorDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
        item["labels"] = torch.tensor(self.labels[idx]) # item["label"]でなくitem["labels"]が正しい！
        return item
    
    def __len__(self):
        return len(self.labels)     

In [6]:
# 保存済みDatasetをpklからロード
with open("../../DataSet/ldcc/dataloader/ds_train.pkl", "rb") as f:
    ds_train = pickle.load(f)
with open("../../DataSet/ldcc/dataloader/ds_valid.pkl", "rb") as f:
    ds_valid = pickle.load(f)
with open("../../DataSet/ldcc/dataloader/ds_test.pkl", "rb") as f:
    ds_test = pickle.load(f)

#### データの準備ここまで
ここからはtransformersを活用\
今回はBertModel（AutoModel）から自前で構築してみる

ポイントは
1. forwardの出力の最初は必ずlossの値にする（Trainerの仕様）
2. loss計算のためにforwardの引数にはlabelが必要
3. その他のloss計算に必要な設定も引数として渡す。\
使わないとしても実行時エラーや学習に誤りが含まれる可能性があるので引数として受け口だけでも用意しておく

In [18]:
"""
ファインチューニング用モデルは読み込むだけ
"""
class BertClassifier(nn.Module):
    def __init__(self, pretrained_model):
        super(BertClassifier, self).__init__()
        
        self.bert = pretrained_model
        self.dropout = nn.Dropout(p=.1)
        self.classifier = nn.Linear(in_features=768, out_features=9) #9カテゴリのクラス分類
        
        # 重み初期化
        nn.init.normal_(self.classifier.weight, std=.02)
        nn.init.normal_(self.classifier.bias, 0)
        
    def forward(
        self, 
        input_ids, 
        labels=None, 
        attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, output_attentions=None,
        output_hidden_states=None, return_dict=None    
    ):
        output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids, 
            position_ids=position_ids,
            head_mask=head_mask, 
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )
        pooler_output = output.pooler_output
        pooler_output = self.dropout(pooler_output)
        output_classifier = self.classifier(pooler_output)
        
        # loss計算
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(output_classifier.view(-1, 9), labels.view(-1))
        
        # 出力はlossが先
        return loss, output_classifier   

In [19]:
model = AutoModel.from_pretrained(pretrained_model_name)
my_model = BertClassifier(model)

loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese/resolve/main/config.json from cache at C:\Users\licht/.cache\huggingface\transformers\d17efb013e5f9c681234d84641372bffa763904845d8a9ec7ecd62faa5ad1c79.abeb707b5d79387dd462e8bfb724637d856e98434b6931c769b8716c6f287258
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file h

#### 学習工程はTrainerを使って定義
* TrainingArgumentsでコンフィグ指定
* Trainerインスタンス作成
    - モデルやデータセットはここで渡す
    - 必要に応じて評価時のメトリクス計算関数をセット（accとかprとかf1とか）
* Trainer.train()で学習

In [20]:
# 評価関数の設定
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [21]:
# TrainingArguments, Trainerを定義
training_args = TrainingArguments(
    output_dir='./mymodel_outputs/',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=False,
    evaluation_strategy='steps',
    eval_steps=50
)

if "trainer" in locals():
    del trainer

trainer = Trainer(
    model=my_model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
# ファインチューニング
%time trainer.train()

C:\Users\licht\anaconda3\envs\py39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5893
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1474


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,2.123700,1.974415,0.335142,0.236424,0.362432,0.311279
100,1.409900,1.156259,0.645862,0.575454,0.653083,0.604892
150,0.815200,0.874353,0.728630,0.680074,0.741021,0.701017
200,0.833900,0.742648,0.763908,0.701993,0.849797,0.733442
250,0.731900,0.506380,0.834464,0.801306,0.864283,0.805699
300,0.425000,0.493393,0.842605,0.835174,0.851145,0.847503
350,0.811800,0.589265,0.823609,0.800960,0.830522,0.804696
400,0.679300,0.422217,0.869742,0.860441,0.869675,0.860072
450,0.253800,0.334007,0.902307,0.893042,0.901446,0.890129
500,0.822500,0.357998,0.891452,0.883330,0.890107,0.886184


***** Running Evaluation *****
  Num examples = 737
  Batch size = 8
C:\Users\licht\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 737
  Batch size = 8
C:\Users\licht\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 737
  Batch size = 8
***** Running Evaluation *****
  Num examples = 737
  Batch size = 8
***** Running Evaluation *****
  Num examples = 737
  Batch size = 8
*****

CPU times: total: 18min 48s
Wall time: 19min 1s


TrainOutput(global_step=1474, training_loss=0.5342346824948225, metrics={'train_runtime': 1141.6535, 'train_samples_per_second': 5.162, 'train_steps_per_second': 1.291, 'total_flos': 0.0, 'train_loss': 0.5342346824948225, 'epoch': 1.0})

In [23]:
# validationでの性能評価
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 737
  Batch size = 8


{'eval_loss': 0.2155521959066391,
 'eval_accuracy': 0.9402985074626866,
 'eval_f1': 0.9314369480683583,
 'eval_precision': 0.9318940677639961,
 'eval_recall': 0.9315479030773096,
 'eval_runtime': 19.7575,
 'eval_samples_per_second': 37.302,
 'eval_steps_per_second': 4.707,
 'epoch': 1.0}

In [ ]:
# ファインチューニングしたモデルをテストデータで性能評価
trainer.evaluate(ds_test)

***** Running Evaluation *****
  Num examples = 737
  Batch size = 8
